In [9]:
import time
import pandas as pd
import zillow_functions as zl
from bs4 import BeautifulSoup

st = zl.zipcodes_list(st_items = ['08817', '08818', '08820', '08837', '08899'])

# Initialize the webdriver.
driver = zl.init_driver('/usr/local/bin/chromedriver')

# Go to www.zillow.com/homes
zl.navigate_to_website(driver, "http://www.zillow.com/homes")

# Click the "buy" button.
zl.click_buy_button(driver)

# Create 11 variables from the scrapped HTML data.
# These variables will make up the final output dataframe.
df = pd.DataFrame({'address' : [], 
                   'bathrooms' : [], 
                   'bedrooms' : [], 
                   'city' : [], 
                   'days_on_zillow' : [], 
                   'price' : [], 
                   'sale_type' : [], 
                   'state' : [], 
                   'sqft' : [], 
                   'url' : [], 
                   'zip' : []})

# Get total number of search terms.
num_search_terms = len(st)

# Start the scraping.
for k in range(num_search_terms):
    # Define search term (must be str object).
    search_term = st[k]

    # Enter search term and execute search.
    if zl.enter_search_term(driver, search_term):
        print("Entering search term number " + str(k+1) + 
              " out of " + str(num_search_terms))
    else:
        print("Search term " + str(k+1) + 
              " failed, moving onto next search term\n***")
        continue
    
    # Check to see if any results were returned from the search.
    # If there were none, move onto the next search.
    if zl.results_test(driver):
        print("Search " + str(search_term) + 
              " returned zero results. Moving onto the next search\n***")
        continue
    
    # Pull the html for each page of search results. Zillow caps results at 
    # 20 pages, each page can contain 26 home listings, thus the cap on home 
    # listings per search is 520.
    raw_data = zl.get_html(driver)
    print(str(len(raw_data)) + " pages of listings found")
    
    # Take the extracted HTML and split it up by individual home listings.
    listings = zl.get_listings(raw_data)
    
    # For each home listing, extract the 11 variables that will populate that 
    # specific observation within the output dataframe.
    for n in range(len(listings)):
        soup = BeautifulSoup(listings[n], "lxml")
        new_obs = []
        
        # List that contains number of beds, baths, and total sqft (and 
        # sometimes price as well).
        card_info = zl.get_card_info(soup)
        
        # Street Address
        new_obs.append(zl.get_street_address(soup))
        
        # Bathrooms
        new_obs.append(zl.get_bathrooms(card_info))
        
        # Bedrooms
        new_obs.append(zl.get_bedrooms(card_info))
        
        # City
        new_obs.append(zl.get_city(soup))
        
        # Days on the Market/Zillow
        new_obs.append(zl.get_days_on_market(soup))
        
        # Price
        new_obs.append(zl.get_price(soup, card_info))
        
        # Sale Type (House for Sale, New Construction, Foreclosure, etc.)
        new_obs.append(zl.get_sale_type(soup))
        
        # Sqft
        new_obs.append(zl.get_sqft(card_info))
        
        # State
        new_obs.append(zl.get_state(soup))
        
        # URL for each house listing
        new_obs.append(zl.get_url(soup))
        
        # Zipcode
        new_obs.append(zl.get_zipcode(soup))
    
        # Append new_obs to df as a new observation
        if len(new_obs) == len(df.columns):
            df.loc[len(df.index)] = new_obs

# Close the webdriver connection.
zl.close_connection(driver)

# Write df to CSV.
columns = ['address', 'city', 'state', 'zip', 'price', 'sqft', 'bedrooms', 
           'bathrooms', 'days_on_zillow', 'sale_type', 'url']
df = df[columns]
dt = time.strftime("%Y-%m-%d") + "_" + time.strftime("%H%M%S")
file_name = str(dt) + ".csv"
df.to_csv(file_name, index = False)

Entering search term number 1 out of 5
14 pages of listings found
329 home listings scraped
***
Entering search term number 2 out of 5
Search 08818 returned zero results. Moving onto the next search
***
Entering search term number 3 out of 5
9 pages of listings found
217 home listings scraped
***
Entering search term number 4 out of 5
6 pages of listings found
143 home listings scraped
***
Entering search term number 5 out of 5
Search 08899 returned zero results. Moving onto the next search
***


In [5]:
df.head


<bound method NDFrame.head of                          address        city state    zip    price  sqft  \
0                90 Montadale Dr   Princeton    NJ  08540  1995000  9362   
1                     2 Acken Ln   Princeton    NJ  08540   369900  1489   
2                   3 Hemlock Ct   Princeton    NJ  08540   814000  4102   
3                 44 Scribner Ct   Princeton    NJ  08540  1249000  3536   
4                61 Cleveland Ln   Princeton    NJ  08540  1450000  3205   
5                  7 Allison Way   Princeton    NJ  08540   628888  3349   
6                     1303 Canal    Franklin    NJ  08540   710000  3207   
7                     20 Taft Ct   Princeton    NJ  08540   385000    NA   
8                  276 Gallup Rd   Princeton    NJ  08540  1255000  3112   
9                   139 Sayre Dr   Princeton    NJ  08540   444444  1355   
10                   3 Hornor Ln   Princeton    NJ  08540   650000  1480   
11                     84 Old Rd   Princeton    NJ  08540 